In [ ]:
import os

In [ ]:
%pwd

'c:\\Users\\pc\\Desktop\\ai\\mlops-datascience\\research'

In [ ]:
os.chdir("../")

In [ ]:
%pwd

'c:\\Users\\pc\\Desktop\\ai\\mlops-datascience'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig: 
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:str
    alpha:float
    l1_ratio:float
    target_column:str

In [ ]:
from src.mlops_pipeline.constants import *
from src.mlops_pipeline.utils.common import read_yaml,create_directories


In [ ]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        print("alphabeta",self.params)
        config= self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([self.config.artifacts_root])
        
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column= schema.name
        )
        
        return model_trainer_config
        
    

In [ ]:
import pandas as pd
import os
from src.mlops_pipeline import logger
from sklearn.linear_model import ElasticNet
import joblib


class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config
        
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y= train_data[self.config.target_column]
        test_y = train_data[self.config.target_column]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x,train_y)
        
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

In [ ]:


STAGE_NAME = "Model Trainer stage"

try:
    config = ConfigurationManager()
    data_validation_config = config.get_model_trainer_config()
    data_validation = ModelTrainer(config=data_validation_config)
    data_validation.train()
    
except Exception as e:
    logger.exception(e)
    raise e

[2025-11-18 11:46:37,324: ERROR: 3119319871 :Argument path_to_yaml of type <class 'list'> to <function read_yaml at 0x0000019AF6467370> does not match annotation type <class 'pathlib.Path'>]
Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_30148\3119319871.py", line 4, in <module>
    config = ConfigurationManager()
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_30148\3719525093.py", line 3, in __init__
    self.config = read_yaml([config_filepath])
  File "c:\Users\pc\Desktop\ai\mlops-datascience\venv\lib\site-packages\ensure\main.py", line 870, in __call__
    raise EnsureError(msg.format(arg=arg, f=self.f, t=templ, valt=type(value)))
ensure.main.EnsureError: Argument path_to_yaml of type <class 'list'> to <function read_yaml at 0x0000019AF6467370> does not match annotation type <class 'pathlib.Path'>


EnsureError: Argument path_to_yaml of type <class 'list'> to <function read_yaml at 0x0000019AF6467370> does not match annotation type <class 'pathlib.Path'>